### 神经语言模型中，p(Wi|W1,...,Wi-1)分布由一个softmax层产生
### tensorflow提供两个函数计算交叉熵：
### tf.nn.softmax_cross_entropy_with_logits和tf.nn.sparse_softmax_cross_entropy_with_logits

In [1]:
# 词汇表大小为3，语料包含两个单词“2 0”

import tensorflow as tf

word_labels = tf.constant([2,0])

predict_logits = tf.constant([[2.0, -1.0, 3.0],
                             [1.0, 0.0, -0.5]])

# 使用tf.nn.sparse_softmax_cross_entropy_with_logits计算交叉熵
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=word_labels, logits=predict_logits)
sess = tf.Session()
sess.run(loss)

/home/wujiaocan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


array([0.32656264, 0.4643688 ], dtype=float32)

In [3]:
word_prob_distribution = tf.constant([[0.0, 0.0, 1.0],
                                     [1.0, 0.0, 0.0]])
loss = tf.nn.softmax_cross_entropy_with_logits(
    labels=word_prob_distribution, logits=predict_logits)

sess.run(loss)

array([0.32656264, 0.4643688 ], dtype=float32)

### PTB数据集预处理

In [1]:
# 将单词转化成模型可以读入的单词序列，将这10000个不同的词汇分别映射到0~9999之间的整数编号
# 按照词频顺序为每个词分配一个编号，然后将词汇表保存到一个独立的covab文件

import codecs
import collections
from operator import itemgetter

RAW_DATA = "./data/ptb.train.txt"  # 训练集数据文件，低频词已经转换成"<unk>"
VOCAB_OUTPUT = "./data/ptb.vocab"  # 输出的词汇表文件

counter = collections.Counter()

with codecs.open(RAW_DATA, "r", encoding="utf-8") as f:
    for line in f:
        for word in line.strip().split():
            counter[word] += 1
            
# 按词频顺序对单词进行排序
sorted_word_to_cnt = sorted(counter.items(), key=itemgetter(1), reverse=True)
sorted_words = [x[0] for x in sorted_word_to_cnt]

# 稍后需要在文本换行处加入句子结束符"<eos>"，预先将其加入词汇表
sorted_words = ["<eos>"] + sorted_words 

with codecs.open(VOCAB_OUTPUT, "w", encoding="utf-8") as file_output:
    for word in sorted_words:
        file_output.write(word + "\n")

In [2]:
# 确定词汇表之后，将训练文件，测试文件等根据词汇文件转化为单词编号，每个单词的编号就是它在词汇文件中的行号

import codecs
import sys

RAW_DATA="./data/ptb.train.txt"  # 原始训练集数据文件
VOCAB = "./data/ptb.vocab"       #  词汇表文件
OUTPUT_DATA = "./data/ptb.train"    # 将单词替换为单词编号后的输出文件

# 读取词汇表，并建立词汇到单词编号的映射
with codecs.open(VOCAB, "r", encoding="utf-8") as f_vocab:
    vocab = [w.strip() for w in f_vocab.readlines()]
word_to_id = {k:v for (k,v) in zip(vocab, range(len(vocab)))}

# 如果出现了被删除的低频词，则替换为"<unk>"
def get_id(word):
    return word_to_id[word] if word in word_to_id else word_to_id["<unk>"]

fin = codecs.open(RAW_DATA, "r", encoding="utf-8")
fout = codecs.open(OUTPUT_DATA, "w", encoding="utf-8")
for line in fin:
    words = line.strip().split() + ["<eos>"]
    # 将每个单词替换为词汇表中的编号
    out_line = " ".join([str(get_id(w)) for w in words]) + "\n"
    fout.write(out_line)
fin.close()
fout.close()

In [13]:
# valid/test文件也将单词转化成数字

import codecs
import sys

RAW_DATA="./data/ptb.test.txt"  # 原始训练集数据文件
VOCAB = "./data/ptb.vocab"       #  词汇表文件
OUTPUT_DATA = "./data/ptb.test"    # 将单词替换为单词编号后的输出文件

# 读取词汇表，并建立词汇到单词编号的映射
with codecs.open(VOCAB, "r", encoding="utf-8") as f_vocab:
    vocab = [w.strip() for w in f_vocab.readlines()]
word_to_id = {k:v for (k,v) in zip(vocab, range(len(vocab)))}

# 如果出现了被删除的低频词，则替换为"<unk>"
def get_id(word):
    return word_to_id[word] if word in word_to_id else word_to_id["<unk>"]

fin = codecs.open(RAW_DATA, "r", encoding="utf-8")
fout = codecs.open(OUTPUT_DATA, "w", encoding="utf-8")
for line in fin:
    words = line.strip().split() + ["<eos>"]
    # 将每个单词替换为词汇表中的编号
    out_line = " ".join([str(get_id(w)) for w in words]) + "\n"
    fout.write(out_line)
fin.close()
fout.close()

In [14]:
import codecs
import sys

RAW_DATA="./data/ptb.valid.txt"  # 原始训练集数据文件
VOCAB = "./data/ptb.vocab"       #  词汇表文件
OUTPUT_DATA = "./data/ptb.valid"    # 将单词替换为单词编号后的输出文件

# 读取词汇表，并建立词汇到单词编号的映射
with codecs.open(VOCAB, "r", encoding="utf-8") as f_vocab:
    vocab = [w.strip() for w in f_vocab.readlines()]
word_to_id = {k:v for (k,v) in zip(vocab, range(len(vocab)))}

# 如果出现了被删除的低频词，则替换为"<unk>"
def get_id(word):
    return word_to_id[word] if word in word_to_id else word_to_id["<unk>"]

fin = codecs.open(RAW_DATA, "r", encoding="utf-8")
fout = codecs.open(OUTPUT_DATA, "w", encoding="utf-8")
for line in fin:
    words = line.strip().split() + ["<eos>"]
    # 将每个单词替换为词汇表中的编号
    out_line = " ".join([str(get_id(w)) for w in words]) + "\n"
    fout.write(out_line)
fin.close()
fout.close()

### PTB数据batching 

In [3]:
import numpy as np
import tensorflow as tf

TRAIN_DATA = "./data/ptb.train"  #使用单词编号便是的训练数据
TRAIN_BATCH_SIZE = 20
TRAIN_NUM_STEP = 35

# 从文件中读取数据，并返回包含单词编号的数据
def read_data(file_path):
    with open(file_path, "r") as fin:
        # 将整个文档读进一个长字符串
        id_string=" ".join([line.strip() for line in fin.readlines()])
    id_list = [int(w) for w in id_string.split()]  # 将读取的单词编号转为整数
    return id_list

def make_batches(id_list, batch_size, num_step):
    # 计算总的batch数量。每个batch包含的单词数量是batch_size * num_step。
    num_batches = (len(id_list)-1) // (batch_size*num_step)
    
    # 数据整理成维度为[batch_size, num_batches*num_step]的二维数组
    data = np.array(id_list[:num_batches * batch_size * num_step])
    data = np.reshape(data, [batch_size, num_batches * num_step])
    # 沿着第二个维度将数据切分成num_batches个batch,存入一个数组
    data_batches = np.split(data, num_batches, axis=1)

    # 重复上述操作，但是每个位置向右移动一位。这里得到的是RNN每一步输出所需要预测的下一个单词。
    label = np.array(id_list[1: num_batches * batch_size * num_step + 1])
    label = np.reshape(label, [batch_size, num_batches * num_step])
    label_batches = np.split(label, num_batches, axis=1)
    # 返回一个长度为num_batches的数组，其中每一项包括一个data矩阵和一个label矩阵
    return list(zip(data_batches, label_batches))

def main():
    train_batches = make_batches(read_data(TRAIN_DATA),
                                TRAIN_BATCH_SIZE, TRAIN_NUM_STEP)
if __name__ == "__main__":
    main()

/home/wujiaocan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
read_data(TRAIN_DATA)

[9970,
 9971,
 9972,
 9973,
 9974,
 9975,
 9976,
 9977,
 9978,
 9979,
 9980,
 9981,
 9982,
 9983,
 9984,
 9985,
 9986,
 9987,
 9988,
 9989,
 9990,
 9991,
 9992,
 9993,
 0,
 8569,
 2,
 3,
 72,
 393,
 33,
 2116,
 1,
 146,
 19,
 6,
 8570,
 275,
 407,
 3,
 0,
 23,
 2,
 13,
 141,
 4,
 2,
 5278,
 1,
 3055,
 1581,
 96,
 0,
 7232,
 2,
 3,
 72,
 393,
 8,
 337,
 141,
 4,
 2468,
 657,
 2158,
 949,
 24,
 521,
 6,
 8570,
 275,
 4,
 39,
 303,
 437,
 3661,
 0,
 6,
 941,
 4,
 3143,
 495,
 262,
 5,
 137,
 5882,
 4219,
 5883,
 30,
 986,
 6,
 240,
 755,
 4,
 1013,
 2765,
 211,
 6,
 96,
 4,
 427,
 4060,
 5,
 14,
 45,
 55,
 3,
 72,
 195,
 1233,
 220,
 0,
 1,
 3143,
 7233,
 2,
 13,
 3919,
 2,
 495,
 14,
 6720,
 1,
 2,
 22,
 113,
 2646,
 7825,
 5,
 14,
 2469,
 5051,
 10,
 462,
 52,
 2980,
 465,
 1233,
 15,
 0,
 2,
 80,
 1,
 167,
 4,
 35,
 2582,
 2,
 65,
 10,
 557,
 5882,
 3555,
 1872,
 663,
 2,
 7,
 27,
 2,
 4219,
 5883,
 7,
 3,
 0,
 366,
 1952,
 3144,
 46,
 220,
 45,
 55,
 6,
 40,
 195,
 1,
 466,
 342,
 129

In [7]:
make_batches(read_data(TRAIN_DATA),TRAIN_BATCH_SIZE, TRAIN_NUM_STEP)[0]

(array([[9970, 9971, 9972, 9973, 9974, 9975, 9976, 9977, 9978, 9979, 9980,
         9981, 9982, 9983, 9984, 9985, 9986, 9987, 9988, 9989, 9990, 9991,
         9992, 9993,    0, 8569,    2,    3,   72,  393,   33, 2116,    1,
          146,   19],
        [  13, 1513,   18, 1446,    1,  844,  236,    1, 1384,    5, 1273,
            7, 1635, 1089, 3842,   17,  380, 1352,    4,  207,    0,    1,
         2600,    4,    1,  261,   13,    5,  335,    1,    2,   16,  767,
         1499,   10],
        [   1, 1123,  646,   46,   20,    2, 1062,   82, 1088,  475,    6,
         1902,    7,    2,    2,    8, 7504,   80,    6,    2, 2142,    7,
         1946,    0, 5660,   82, 8878,  557,  548,    2,   22, 8953,    8,
          539,    2],
        [ 385,   14,   93,   25, 1018,    5,  254,  170,   10,  207,    0,
           54, 1451, 1258,   22, 1659,   15,    1,  469,   42,   45,   55,
         1844,    1,   37,    9,  207,    4,  511,   12,    3,   48,    0,
           14,   59],
        [  5

#### embedding层

In [ ]:
embedding = tf.get_variable("embedding", [VOCAB_SIZE, EMB_SIZE])
# 输出的矩阵比输入数据多一个维度，新增维度的大小是EMB_SIZE
# 一般input_data的维度是batch_size * num_step
# 而输出的input_embedding维度是batch_size * num_step * EMB_SIZE
input_embedding = tf.nn.embedding_lookup(embedding, input_data)

#### softmax层

In [ ]:
# 定义线性映射用到的参数
# HIDDEN_SIZE是循环神经网络的隐藏状态维度，VOCAB_SIZE是词汇表的大小
weight = tf.get_variable("weight", [HIDDEN_SIZE, COVAB_SIZE])
bias = tf.get_variable("bias", [VOCAB_SIZE])
# 计算线性映射
# output是RNN的输出，维度为[batch_size * num_steps, HIDDEN_SIZE]
logits = tf.nn.bias_add(tf.matmul(output, weight), bias)

In [ ]:
# probs的维度与logits的维度相同
probs = tf.nn.softmax(logits)

In [ ]:
# label是一个大小为[batch_size * num_steps]的一组数组，包含每个位置正确的单词编号
# logits维度是[batch_size * num_steps, HIDDEN_SIZE]
# loss维度与labels相同，代表每个位置上的log perplexity
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.reshape(self.targets, [-1]), logits=logitts)

In [ ]:
## embedding

embedding = tf.get_varibale("embedding", [VAOCAB_SIZE, EMB_SIZE])
input_embedding = tf.nn.embedding_lookup(embedding,input_data)

In [ ]:
## softmax

weight = tf.get_varibale("weight", [HIDDEN_SIZE, VOCAB_SIZE])
bias = tf.get_variable("bias", [VOCAB_SIZE])

logits = tf.nn.bias_add(tf.matmul(output, weight), bias)

probs = tf.nn.softmax(logits)
